<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1/TP1_Parte2_Python_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
%%writefile tp1-parte2.py
import sys
import os
import random
from threading import Thread
MIN_MATRIX_SIZE = 5
MAX_MATRIX_SIZE = 20
MIN_MATRIX_CELL_VALUE = -32
MAX_MATRIX_CELL_VALUE = 32

def validate_args(args):
    if not len(args) == 2:
        print("Cantidad incorrecta de argumentos")
        os._exit(1)
    size = int(args[1])
    if size > MAX_MATRIX_SIZE or size < MIN_MATRIX_SIZE:
        print("El tamaño debe estar entre {min} y {max}".format(
          min=MIN_MATRIX_SIZE,
          max=MAX_MATRIX_SIZE))
        os._exit(1)
    return size

def matrix_generator(size):
    matrix_result=[]
    for i in range(0, size):
        matrix_row=[]
        for j in range(0, size):
            matrix_row.append(random.randint(
              MIN_MATRIX_CELL_VALUE, MAX_MATRIX_CELL_VALUE))
        matrix_result.append(matrix_row)
    return matrix_result

def print_matrix(alias,matrix):
    print("\nMatrix: {alias}".format(alias=alias))
    for row in matrix:
        for col in row:
            print("\t{col}\t".format(col=col),end="")
        print("\n",end="")

def secuential_mult(size, matrix_a, matrix_b):
    matrix_result = []
    for i in range(0, size):
        row_result = [0] * size
        for j in range(0,size):
            for k in range(0,size):
                row_result[j] += matrix_a[i][k] * matrix_b[k][j]
        matrix_result.append(row_result)
    return matrix_result

## Funcion que ejecuta el thread
def multiply_row(size, matrix_a, matrix_b, row_result, rowIndex):
    for j in range(0,size):
        for k in range(0,size):
            row_result[j]+= matrix_a[rowIndex][k]*matrix_b[k][j]

## Genera un thread por cada fila encargado de ejecutar la multiplicacion y generar la fila resultante
def concurrent_mult(size, matrix_a, matrix_b):
    matrix_result = []
    threads = []

    for i in range(0, size):
        row_result = [0] * size
        thread = Thread(
          target=multiply_row,
          args=(size, matrix_a, matrix_b, row_result, i))
        thread.start()
        threads.append(thread)
        matrix_result.append(row_result)

    for thread in threads:
        thread.join()

    return matrix_result

def compare_matrix(size, matrix_a, matrix_b):
    for i in range(0, size):
        for j in range(0, size):
            if matrix_a[i][j] != matrix_b[i][j]:
                return False
    return True

def main():
    size = validate_args(sys.argv)
    matrix_a = matrix_generator(size)
    matrix_b = matrix_generator(size)
    matrix_secuential = secuential_mult(size,matrix_a,matrix_b)
    matrix_concurrent = concurrent_mult(size,matrix_a,matrix_b)

    print_matrix("A", matrix_a)
    print_matrix("B", matrix_b)
    print_matrix("Secuencial", matrix_secuential)
    print_matrix("Concurrente", matrix_concurrent)

    ## Determinamos si las matrices son iguales
    result_compare = (
      "iguales" 
      if compare_matrix(size, matrix_secuential, matrix_concurrent)
      else "distintas")
    print("Las matrices concurrente y secuencial son {compare}".format(
      compare=result_compare))

main()

Overwriting tp1-parte2.py


In [17]:
!python3 tp1-parte2.py 1

El tamaño debe estar entre 5 y 20


In [18]:
!python3 tp1-parte2.py 7


Matrix: A
	4		16		2		-3		-17		-3		1	
	-10		-32		18		30		2		-30		4	
	-20		0		14		30		3		15		26	
	-19		-20		-10		29		1		-7		-29	
	-29		4		8		-19		-28		-31		29	
	29		-6		26		18		-18		-4		19	
	6		23		-7		5		21		-19		-15	

Matrix: B
	23		29		-15		27		9		23		23	
	19		13		-21		10		15		-32		-20	
	-5		31		3		25		2		26		-18	
	2		-7		9		18		2		26		20	
	-26		22		-14		-5		7		-19		-32	
	-30		24		-17		12		25		2		-3	
	30		-18		9		18		3		3		15	

Matrix: Secuencial
	942		-57		-119		331		83		-126		244	
	100		-1106		1664		102		-1198		1956		772	
	-218		-398		549		983		382		735		137	
	-1395		-948		780		-1052		-688		577		277	
	1859		-2290		1384		-595		-1107		-524		153	
	1617		609		422		2081		90		2394		1552	
	194		497		-655		-296		22		-1132		-936	

Matrix: Concurrente
	942		-57		-119		331		83		-126		244	
	100		-1106		1664		102		-1198		1956		772	
	-218		-398		549		983		382		735		137	
	-1395		-948		780		-1052		-688		577		277	
	1859		-2290		1384		-595		-1107		-524		153	
	1617		609		422		2081		90		2394		1552	
	1